In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from math import sqrt

In [32]:
train = pd.read_csv("../data/schemas/warm_up/TrainSet.csv", sep=",")
validation = pd.read_csv("../data/schemas/warm_up/ValidationSet.csv", sep=",")
test = pd.read_csv("../data/schemas/warm_up/TestSet.csv", sep=",")

In [33]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4   # For feature selection

In [34]:
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[0]])) > THRESHOLD: #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])
    
if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
print(independent_variables)

['Hour_1', 'Hour_2', 'Hour_3', 'Occupant Count (people)_3', '12h Outdoor Drybulb Temperature (C)', '24h Outdoor Drybulb Temperature (C)', '12h Outdoor Relative Humidity (%)', '6h Direct Solar Radiation (W/m2)', '12h Direct Solar Radiation (W/m2)']


In [35]:
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

print(X_train_default.shape)
print(X_val_default.shape)
print(X_test_default.shape)

(576, 9)
(96, 9)
(96, 9)


In [36]:
# Function to prepare the data into batches that will be passed into the model

def create_sequences(input_data, output_data, timesteps_input, timesteps_output):
    sequences, targets = [], []
    for i in range(len(input_data) - timesteps_input - timesteps_output + 1):
        seq = input_data[i:i + timesteps_input]
        target = output_data[i + timesteps_input: i + timesteps_input + timesteps_output]
        sequences.append(seq)
        targets.append(target)

    return np.array(sequences), np.array(targets)

In [37]:
X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

print(f"X_train = {X_train.shape}, Y_train = {Y_train.shape}\n"
      f"X_val = {X_val.shape}, Y_val = {Y_val.shape}\n"
      f"X_test = {X_test.shape}, Y_test = {Y_test.shape}")

X_train = (505, 48, 9), Y_train = (505, 24, 2)
X_val = (25, 48, 9), Y_val = (25, 24, 2)
X_test = (25, 48, 9), Y_test = (25, 24, 2)


In [38]:
print(Y_train[:, :, 0])

[[0.59739428 0.49429981 0.38708765 ... 0.73547528 0.69513525 0.71729728]
 [0.49429981 0.38708765 0.28880037 ... 0.69513525 0.71729728 0.6633147 ]
 [0.38708765 0.28880037 0.2346659  ... 0.71729728 0.6633147  0.55539687]
 ...
 [0.47416969 0.39741721 0.28594384 ... 0.4397153  0.45619247 0.45482902]
 [0.39741721 0.28594384 0.16805223 ... 0.45619247 0.45482902 0.41836388]
 [0.28594384 0.16805223 0.12974031 ... 0.45482902 0.41836388 0.39162712]]


In [39]:
RMSE_model_1 = []
RMSE_model_2 = []

In [40]:
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras import layers, models

In [41]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(98, activation='tanh', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(48, activation='tanh', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(24, activation='tanh', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(96, activation='tanh', return_sequences=True)(input_layer)
dense5 = layers.LSTM(48, activation='tanh', return_sequences=True)(dense4)
dense6 = layers.LSTM(24, activation='tanh', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 11s 607ms/step - loss: 0.5801 - output_variable1_loss: 0.3979 - output_variable2_loss: 0.1821 - val_loss: 0.3422 - val_output_variable1_loss: 0.1739 - val_output_variable2_loss: 0.1683
Epoch 2/100
4/4 [==============================] - 0s 57ms/step - loss: 0.4472 - output_variable1_loss: 0.2990 - output_variable2_loss: 0.1482 - val_loss: 0.2841 - val_output_variable1_loss: 0.1381 - val_output_variable2_loss: 0.1460
Epoch 3/100
4/4 [==============================] - 0s 54ms/step - loss: 0.3608 - output_variable1_loss: 0.2281 - output_variable2_loss: 0.1327 - val_loss: 0.2443 - val_output_variable1_loss: 0.1081 - val_output_variable2_loss: 0.1362
Epoch 4/100
4/4 [==============================] - 0s 56ms/step - loss: 0.2965 - output_variable1_loss: 0.1717 - output_variable2_loss: 0.1248 - val_loss: 0.2198 - val_output_variable1_loss: 0.0929 - val_output_variable2_loss: 0.1269
Epoch 5/100
4/4 [==============================] - 0s 56ms/ste

In [42]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(98, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(48, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(24, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(96, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(48, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(24, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 6s 276ms/step - loss: 0.5870 - output_variable1_loss: 0.3932 - output_variable2_loss: 0.1938 - val_loss: 0.4010 - val_output_variable1_loss: 0.2036 - val_output_variable2_loss: 0.1974
Epoch 2/100
4/4 [==============================] - 0s 56ms/step - loss: 0.5324 - output_variable1_loss: 0.3563 - output_variable2_loss: 0.1762 - val_loss: 0.3492 - val_output_variable1_loss: 0.1681 - val_output_variable2_loss: 0.1811
Epoch 3/100
4/4 [==============================] - 0s 54ms/step - loss: 0.4648 - output_variable1_loss: 0.3004 - output_variable2_loss: 0.1644 - val_loss: 0.3268 - val_output_variable1_loss: 0.1562 - val_output_variable2_loss: 0.1706
Epoch 4/100
4/4 [==============================] - 0s 54ms/step - loss: 0.4195 - output_variable1_loss: 0.2637 - output_variable2_loss: 0.1558 - val_loss: 0.3019 - val_output_variable1_loss: 0.1372 - val_output_variable2_loss: 0.1647
Epoch 5/100
4/4 [==============================] - 0s 54ms/step

In [43]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(240, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(120, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(240, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(120, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 7s 329ms/step - loss: 0.5780 - output_variable1_loss: 0.3865 - output_variable2_loss: 0.1915 - val_loss: 0.3572 - val_output_variable1_loss: 0.1717 - val_output_variable2_loss: 0.1856
Epoch 2/100
4/4 [==============================] - 0s 104ms/step - loss: 0.5115 - output_variable1_loss: 0.3472 - output_variable2_loss: 0.1643 - val_loss: 0.3008 - val_output_variable1_loss: 0.1408 - val_output_variable2_loss: 0.1601
Epoch 3/100
4/4 [==============================] - 0s 118ms/step - loss: 0.4170 - output_variable1_loss: 0.2682 - output_variable2_loss: 0.1488 - val_loss: 0.2985 - val_output_variable1_loss: 0.1490 - val_output_variable2_loss: 0.1495
Epoch 4/100
4/4 [==============================] - 0s 103ms/step - loss: 0.4248 - output_variable1_loss: 0.2870 - output_variable2_loss: 0.1378 - val_loss: 0.2807 - val_output_variable1_loss: 0.1423 - val_output_variable2_loss: 0.1384
Epoch 5/100
4/4 [==============================] - 0s 110ms/

In [44]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
dense1 = layers.LSTM(480, activation='relu', return_sequences=True)(input_layer)
dense2 = layers.LSTM(240, activation='relu', return_sequences=True)(dense1)
dense3 = layers.LSTM(120, activation='relu', return_sequences=True)(dense2)
dense9 = layers.LSTM(48, activation='relu', return_sequences=False)(dense3)
output_variable1 = layers.Dense(24, name='output_variable1')(dense9)

# Dense layers for variable 2
dense4 = layers.LSTM(480, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(240, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(120, activation='relu', return_sequences=True)(dense5)
dense7 = layers.LSTM(48, activation='relu', return_sequences=False)(dense6)
output_variable2 = layers.Dense(24, name='output_variable2')(dense7)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=200, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/200
4/4 [==============================] - 11s 662ms/step - loss: 0.5802 - output_variable1_loss: 0.3871 - output_variable2_loss: 0.1931 - val_loss: 0.5017 - val_output_variable1_loss: 0.3253 - val_output_variable2_loss: 0.1763
Epoch 2/200
4/4 [==============================] - 1s 367ms/step - loss: 0.4580 - output_variable1_loss: 0.2949 - output_variable2_loss: 0.1632 - val_loss: 0.3290 - val_output_variable1_loss: 0.1687 - val_output_variable2_loss: 0.1603
Epoch 3/200
4/4 [==============================] - 2s 399ms/step - loss: 0.4645 - output_variable1_loss: 0.3165 - output_variable2_loss: 0.1480 - val_loss: 0.2922 - val_output_variable1_loss: 0.1426 - val_output_variable2_loss: 0.1496
Epoch 4/200
4/4 [==============================] - 1s 386ms/step - loss: 0.3862 - output_variable1_loss: 0.2477 - output_variable2_loss: 0.1385 - val_loss: 0.2437 - val_output_variable1_loss: 0.1021 - val_output_variable2_loss: 0.1416
Epoch 5/200
4/4 [==============================] - 1s 377ms

In [45]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(60, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(48, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(24, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(60, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(48, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(24, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 6s 277ms/step - loss: 0.6011 - output_variable1_loss: 0.4007 - output_variable2_loss: 0.2004 - val_loss: 0.4235 - val_output_variable1_loss: 0.2114 - val_output_variable2_loss: 0.2121
Epoch 2/100
4/4 [==============================] - 0s 49ms/step - loss: 0.5678 - output_variable1_loss: 0.3743 - output_variable2_loss: 0.1935 - val_loss: 0.3937 - val_output_variable1_loss: 0.1879 - val_output_variable2_loss: 0.2058
Epoch 3/100
4/4 [==============================] - 0s 53ms/step - loss: 0.5205 - output_variable1_loss: 0.3332 - output_variable2_loss: 0.1873 - val_loss: 0.3421 - val_output_variable1_loss: 0.1465 - val_output_variable2_loss: 0.1956
Epoch 4/100
4/4 [==============================] - 0s 51ms/step - loss: 0.4371 - output_variable1_loss: 0.2613 - output_variable2_loss: 0.1758 - val_loss: 0.3270 - val_output_variable1_loss: 0.1517 - val_output_variable2_loss: 0.1753
Epoch 5/100
4/4 [==============================] - 0s 53ms/step

In [46]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 8) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 6s 272ms/step - loss: 0.5870 - output_variable1_loss: 0.3966 - output_variable2_loss: 0.1904 - val_loss: 0.3847 - val_output_variable1_loss: 0.1938 - val_output_variable2_loss: 0.1909
Epoch 2/100
4/4 [==============================] - 0s 55ms/step - loss: 0.5017 - output_variable1_loss: 0.3331 - output_variable2_loss: 0.1686 - val_loss: 0.2631 - val_output_variable1_loss: 0.0939 - val_output_variable2_loss: 0.1691
Epoch 3/100
4/4 [==============================] - 0s 56ms/step - loss: 0.6510 - output_variable1_loss: 0.5005 - output_variable2_loss: 0.1504 - val_loss: 0.2483 - val_output_variable1_loss: 0.0996 - val_output_variable2_loss: 0.1487
Epoch 4/100
4/4 [==============================] - 0s 55ms/step - loss: 0.3712 - output_variable1_loss: 0.2358 - output_variable2_loss: 0.1354 - val_loss: 0.2723 - val_output_variable1_loss: 0.1339 - val_output_variable2_loss: 0.1385
Epoch 5/100
4/4 [==============================] - 0s 60ms/step

In [47]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 8) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=200, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/200
4/4 [==============================] - 7s 272ms/step - loss: 0.5983 - output_variable1_loss: 0.4085 - output_variable2_loss: 0.1898 - val_loss: 0.4021 - val_output_variable1_loss: 0.2186 - val_output_variable2_loss: 0.1835
Epoch 2/200
4/4 [==============================] - 0s 57ms/step - loss: 0.5425 - output_variable1_loss: 0.3852 - output_variable2_loss: 0.1574 - val_loss: 0.3841 - val_output_variable1_loss: 0.1933 - val_output_variable2_loss: 0.1907
Epoch 3/200
4/4 [==============================] - 0s 57ms/step - loss: 0.4880 - output_variable1_loss: 0.3337 - output_variable2_loss: 0.1543 - val_loss: 0.2821 - val_output_variable1_loss: 0.1308 - val_output_variable2_loss: 0.1513
Epoch 4/200
4/4 [==============================] - 0s 56ms/step - loss: 0.4458 - output_variable1_loss: 0.3058 - output_variable2_loss: 0.1400 - val_loss: 0.2632 - val_output_variable1_loss: 0.1108 - val_output_variable2_loss: 0.1525
Epoch 5/200
4/4 [==============================] - 0s 61ms/step

In [49]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
dense1 = layers.LSTM(96, activation='relu', return_sequences=True)(input_layer)
dense2 = layers.LSTM(48, activation='relu', return_sequences=False)(dense1)
output_variable1 = layers.Dense(24, name='output_variable1')(dense2)

# Dense layers for variable 2
dense4 = layers.LSTM(96, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(48, activation='relu', return_sequences=False)(dense4)
output_variable2 = layers.Dense(24, name='output_variable2')(dense5)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 6s 187ms/step - loss: 0.6030 - output_variable1_loss: 0.4134 - output_variable2_loss: 0.1896 - val_loss: 0.3945 - val_output_variable1_loss: 0.2056 - val_output_variable2_loss: 0.1889
Epoch 2/100
4/4 [==============================] - 0s 42ms/step - loss: 0.5225 - output_variable1_loss: 0.3579 - output_variable2_loss: 0.1646 - val_loss: 0.3262 - val_output_variable1_loss: 0.1680 - val_output_variable2_loss: 0.1582
Epoch 3/100
4/4 [==============================] - 0s 40ms/step - loss: 0.4253 - output_variable1_loss: 0.2856 - output_variable2_loss: 0.1398 - val_loss: 0.3475 - val_output_variable1_loss: 0.2136 - val_output_variable2_loss: 0.1339
Epoch 4/100
4/4 [==============================] - 0s 45ms/step - loss: 0.3564 - output_variable1_loss: 0.2340 - output_variable2_loss: 0.1224 - val_loss: 0.2407 - val_output_variable1_loss: 0.1183 - val_output_variable2_loss: 0.1224
Epoch 5/100
4/4 [==============================] - 0s 40ms/step

In [50]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.5  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 7s 293ms/step - loss: 0.6008 - output_variable1_loss: 0.4065 - output_variable2_loss: 0.1943 - val_loss: 0.4196 - val_output_variable1_loss: 0.2189 - val_output_variable2_loss: 0.2007
Epoch 2/100
4/4 [==============================] - 0s 57ms/step - loss: 0.5637 - output_variable1_loss: 0.3858 - output_variable2_loss: 0.1779 - val_loss: 0.3666 - val_output_variable1_loss: 0.1949 - val_output_variable2_loss: 0.1716
Epoch 3/100
4/4 [==============================] - 0s 60ms/step - loss: 0.4843 - output_variable1_loss: 0.3380 - output_variable2_loss: 0.1463 - val_loss: 0.2929 - val_output_variable1_loss: 0.1492 - val_output_variable2_loss: 0.1437
Epoch 4/100
4/4 [==============================] - 0s 65ms/step - loss: 0.4285 - output_variable1_loss: 0.2949 - output_variable2_loss: 0.1336 - val_loss: 0.2742 - val_output_variable1_loss: 0.1263 - val_output_variable2_loss: 0.1479
Epoch 5/100
4/4 [==============================] - 0s 60ms/step

In [51]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.45  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 6s 269ms/step - loss: 0.5905 - output_variable1_loss: 0.3948 - output_variable2_loss: 0.1958 - val_loss: 0.4006 - val_output_variable1_loss: 0.1947 - val_output_variable2_loss: 0.2059
Epoch 2/100
4/4 [==============================] - 0s 55ms/step - loss: 0.5165 - output_variable1_loss: 0.3323 - output_variable2_loss: 0.1842 - val_loss: 0.3421 - val_output_variable1_loss: 0.1571 - val_output_variable2_loss: 0.1851
Epoch 3/100
4/4 [==============================] - 0s 56ms/step - loss: 0.3980 - output_variable1_loss: 0.2330 - output_variable2_loss: 0.1650 - val_loss: 0.2678 - val_output_variable1_loss: 0.1041 - val_output_variable2_loss: 0.1637
Epoch 4/100
4/4 [==============================] - 0s 59ms/step - loss: 0.3104 - output_variable1_loss: 0.1607 - output_variable2_loss: 0.1497 - val_loss: 0.2435 - val_output_variable1_loss: 0.0878 - val_output_variable2_loss: 0.1557
Epoch 5/100
4/4 [==============================] - 0s 58ms/step

In [52]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.35  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 7s 276ms/step - loss: 0.5919 - output_variable1_loss: 0.3963 - output_variable2_loss: 0.1956 - val_loss: 0.3934 - val_output_variable1_loss: 0.1948 - val_output_variable2_loss: 0.1986
Epoch 2/100
4/4 [==============================] - 0s 56ms/step - loss: 0.5123 - output_variable1_loss: 0.3362 - output_variable2_loss: 0.1761 - val_loss: 0.9390 - val_output_variable1_loss: 0.7741 - val_output_variable2_loss: 0.1649
Epoch 3/100
4/4 [==============================] - 0s 56ms/step - loss: 0.5634 - output_variable1_loss: 0.4093 - output_variable2_loss: 0.1541 - val_loss: 0.2859 - val_output_variable1_loss: 0.1363 - val_output_variable2_loss: 0.1496
Epoch 4/100
4/4 [==============================] - 0s 56ms/step - loss: 0.4044 - output_variable1_loss: 0.2668 - output_variable2_loss: 0.1376 - val_loss: 0.2673 - val_output_variable1_loss: 0.1300 - val_output_variable2_loss: 0.1373
Epoch 5/100
4/4 [==============================] - 0s 55ms/step

In [53]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.3  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 6s 296ms/step - loss: 0.6001 - output_variable1_loss: 0.4108 - output_variable2_loss: 0.1893 - val_loss: 0.4049 - val_output_variable1_loss: 0.2199 - val_output_variable2_loss: 0.1850
Epoch 2/100
4/4 [==============================] - 0s 56ms/step - loss: 0.5465 - output_variable1_loss: 0.3811 - output_variable2_loss: 0.1654 - val_loss: 0.3462 - val_output_variable1_loss: 0.1809 - val_output_variable2_loss: 0.1653
Epoch 3/100
4/4 [==============================] - 0s 61ms/step - loss: 0.4385 - output_variable1_loss: 0.2895 - output_variable2_loss: 0.1490 - val_loss: 6.2464 - val_output_variable1_loss: 6.0912 - val_output_variable2_loss: 0.1552
Epoch 4/100
4/4 [==============================] - 0s 73ms/step - loss: 1.9093 - output_variable1_loss: 1.7685 - output_variable2_loss: 0.1408 - val_loss: 0.2646 - val_output_variable1_loss: 0.1152 - val_output_variable2_loss: 0.1494
Epoch 5/100
4/4 [==============================] - 0s 75ms/step

In [54]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.25  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 7s 277ms/step - loss: 0.5849 - output_variable1_loss: 0.3916 - output_variable2_loss: 0.1933 - val_loss: 0.3633 - val_output_variable1_loss: 0.1694 - val_output_variable2_loss: 0.1939
Epoch 2/100
4/4 [==============================] - 0s 62ms/step - loss: 0.4587 - output_variable1_loss: 0.2830 - output_variable2_loss: 0.1757 - val_loss: 0.3221 - val_output_variable1_loss: 0.1434 - val_output_variable2_loss: 0.1786
Epoch 3/100
4/4 [==============================] - 0s 66ms/step - loss: 0.3602 - output_variable1_loss: 0.1980 - output_variable2_loss: 0.1622 - val_loss: 0.5331 - val_output_variable1_loss: 0.3663 - val_output_variable2_loss: 0.1668
Epoch 4/100
4/4 [==============================] - 0s 66ms/step - loss: 0.3088 - output_variable1_loss: 0.1558 - output_variable2_loss: 0.1530 - val_loss: 0.2391 - val_output_variable1_loss: 0.0844 - val_output_variable2_loss: 0.1547
Epoch 5/100
4/4 [==============================] - 0s 64ms/step

In [55]:
print(RMSE_model_1)
print(RMSE_model_2)

#LSTM

[0.1455930188603446, 0.1490835025288306, 0.1894332047266111, 0.155278305354434, 0.2754457616266509, 0.12684163122930464, 0.15297675508995734, 0.20003717264708776, 0.10988670009484779, 0.12654859808476465, 0.1923043073643966, 0.1710723574537041, 0.20804791387577118]
[0.08069823931631688, 0.09342870438981896, 0.08894496277491347, 0.11448260132550502, 0.06771893235360013, 0.05024733013340825, 0.10885519998541643, 0.11951677421810032, 0.1463912344970114, 0.1284874060452856, 0.04477812863954874, 0.04097768493457772, 0.031222598600959106]


In [56]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
"""    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=150, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
"""

history = model.fit(X_train, y={"output_variable1": Y_train[:, :, 0], 
                                "output_variable2": Y_train[:, :, 1]}, epochs=200, verbose=1)

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/200
16/16 [==============================] - 7s 40ms/step - loss: 0.4983 - output_variable1_loss: 0.3318 - output_variable2_loss: 0.1665
Epoch 2/200
16/16 [==============================] - 1s 42ms/step - loss: 0.3589 - output_variable1_loss: 0.2307 - output_variable2_loss: 0.1283
Epoch 3/200
16/16 [==============================] - 1s 42ms/step - loss: 0.2067 - output_variable1_loss: 0.1171 - output_variable2_loss: 0.0896
Epoch 4/200
16/16 [==============================] - 1s 42ms/step - loss: 0.1448 - output_variable1_loss: 0.0678 - output_variable2_loss: 0.0769
Epoch 5/200
16/16 [==============================] - 1s 42ms/step - loss: 0.0866 - output_variable1_loss: 0.0463 - output_variable2_loss: 0.0403
Epoch 6/200
16/16 [==============================] - 1s 44ms/step - loss: 0.0664 - output_variable1_loss: 0.0417 - output_variable2_loss: 0.0246
Epoch 7/200
16/16 [==============================] - 1s 45ms/step - loss: 0.0587 - output_variable1_loss: 0.0406 - output_variable

In [57]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Dense layers for variable 1
lstm1 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
lstm2 = layers.LSTM(60, activation='relu', return_sequences=True)(lstm1)
lstm3 = layers.LSTM(48, activation='relu', return_sequences=False)(lstm2)
output_variable1 = layers.Dense(24, name='output_variable1')(lstm3)

# Dense layers for variable 2
dense4 = layers.LSTM(120, activation='relu', return_sequences=True)(input_layer)
dense5 = layers.LSTM(60, activation='relu', return_sequences=True)(dense4)
dense6 = layers.LSTM(48, activation='relu', return_sequences=False)(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
   
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=150, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
"""

history = model.fit(X_train, y={"output_variable1": Y_train[:, :, 0], 
                                "output_variable2": Y_train[:, :, 1]}, epochs=200, verbose=1)
""" 

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/150
4/4 [==============================] - 7s 587ms/step - loss: 0.5751 - output_variable1_loss: 0.3818 - output_variable2_loss: 0.1933 - val_loss: 0.3671 - val_output_variable1_loss: 0.1702 - val_output_variable2_loss: 0.1969
Epoch 2/150
4/4 [==============================] - 0s 52ms/step - loss: 0.4522 - output_variable1_loss: 0.2786 - output_variable2_loss: 0.1736 - val_loss: 0.3115 - val_output_variable1_loss: 0.1392 - val_output_variable2_loss: 0.1724
Epoch 3/150
4/4 [==============================] - 0s 54ms/step - loss: 0.3384 - output_variable1_loss: 0.1815 - output_variable2_loss: 0.1569 - val_loss: 0.2495 - val_output_variable1_loss: 0.0892 - val_output_variable2_loss: 0.1603
Epoch 4/150
4/4 [==============================] - 0s 53ms/step - loss: 0.2780 - output_variable1_loss: 0.1326 - output_variable2_loss: 0.1454 - val_loss: 0.2623 - val_output_variable1_loss: 0.1122 - val_output_variable2_loss: 0.1501
Epoch 5/150
4/4 [==============================] - 0s 62ms/step